In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import librosa

In [11]:
def nu_wave_model(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Convolutional layers
    conv1 = layers.Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu')(inputs)
    conv2 = layers.Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu')(conv1)

    # Recurrent layers
    rnn1 = layers.Bidirectional(layers.GRU(256, return_sequences=True))(conv2)
    rnn2 = layers.Bidirectional(layers.GRU(256, return_sequences=True))(rnn1)

    # Upsampling layers
    upsample1 = layers.Conv1DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu')(rnn2)
    upsample2 = layers.Conv1DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(upsample1)

    # Output layer
    output = layers.Conv1D(1, kernel_size=3, strides=1, padding='same', activation='sigmoid')(upsample2)

    return Model(inputs, output)


In [12]:
def load_audio_file(file_path):
    audio_data, sampling_rate = librosa.load(file_path, sr=None)
    return audio_data, sampling_rate

In [9]:
def main():
    audio_file_path = "/content/samples_msp_4_msp.4.4.lr.wav"

    audio_data, _ = load_audio_file(audio_file_path)

    audio_data /= np.max(np.abs(audio_data))

    input_shape = (None, 1)
    model = nu_wave_model(input_shape)

    plt.figure(figsize=(10, 4))
    plt.title('Input Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.plot(np.arange(len(audio_data)) / len(audio_data), audio_data)
    plt.grid(True)
    plt.show()

    audio_data = np.expand_dims(audio_data, axis=0)
    audio_data = np.expand_dims(audio_data, axis=-1)

    prediction = model.predict(audio_data)

    plt.figure(figsize=(10, 4))
    plt.title('Predicted Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.plot(np.arange(len(prediction.squeeze())) / len(prediction.squeeze()), prediction.squeeze())
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()

C:\Users\email\AppData\Local\Temp\ipykernel_15492\919982993.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sampling_rate = librosa.load(file_path, sr=None)
C:\Users\email\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/content/samples_msp_4_msp.4.4.lr.wav'